# Revenue Management without Demand Forecasting
Access paper: https://arxiv.org/abs/2304.07391

In [ ]:
import numpy as np
import math, statistics
import time
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from scipy.optimize import minimize_scalar
import multiprocessing as mp

## Optimal Benchmark: Generate Optimal Bid Price using DP

In [ ]:
# Define utility functions
def p0_func(t):
    return 100  # Minimum price

def alpha_func(t):
    return 50  # Mean willingness-to-pay of customers

def pmax_func(t, lambda_val=3):
    pmax = p0_func(t) - alpha_func(t) * np.log(85 / (lambda_val * 300))
    pmax_rounded = np.ceil(pmax / 50) * 50 # Round up the result to the nearest integer greater than or equal to the floating-point number.
    # return int(pmax_rounded)
    return 500

def Pw_func(p, t):
    return np.exp(-(p - p0_func(t)) / alpha_func(t))  # Exponential purchase probability

def lambda_func(t):
    return 3  # Constant arrival rate

In [ ]:
def optimal_bid_prices(C, T, p0_func, pmax_func, lambda_func, Pw_func, delta_t):
    # Step 1: Define state space
    X = np.arange(0, C+1)  # Remaining inventory levels
    t = np.arange(0, T+1)  # Time periods

    # Step 2: Initialize value function matrix, optimal price matrix, and bid price matrix
    V = np.zeros((C+1, T+1))  # V(x,t)
    p_star = np.zeros((C+1, T+1))  # Optimal price for each state (x,t)
    b_star = np.zeros((C+1, T+1))  # Bid price for each state (x,t)

    p_star_alternative = np.zeros((C+1, T+1))  # Optimal price for each state (x,t) using Equation (6)

    # Define the boundary conditions for value function
    V[:, 0] = 0  # V(x, 0) = 0 for all x
    V[0, :] = 0  # V(0, t) = 0 for all t

    # Step 3: Solve the dynamic programming problem using backward induction
    for t_idx in range(1, T+1):
        for x_idx in range(1, C+1):
            # Define the revenue function to be maximized
            def revenue_func(p):
                return -1 * lambda_func(t_idx) * delta_t * Pw_func(p, t_idx) * (p - (V[x_idx, t_idx-1] - V[x_idx-1, t_idx-1]))

            # Use golden-section search to find the optimal price
            bounds = (p0_func(t_idx), pmax_func(t_idx))
            result = minimize_scalar(revenue_func, bounds=bounds, method='bounded')
            optimal_price = result.x
            max_val = -1 * result.fun + V[x_idx, t_idx-1]

            V[x_idx, t_idx] = max_val
            p_star[x_idx, t_idx] = optimal_price
            b_star[x_idx, t_idx] = V[x_idx, t_idx] - V[x_idx-1, t_idx]

            # Calculate the optimal price using equation (6)
            p_star_alternative[x_idx, t_idx] = max(p0_func(t_idx), alpha_func(t_idx) + b_star[x_idx, t_idx-1])

    return V, b_star, p_star, p_star_alternative

Explanation:
- The `optimal_bid_prices` function takes the capacity `C`, time horizon `T`, arrival rate function `lambda_func`, purchase probability function `Pw_func`, and time interval `delta_t` as inputs.
- We define the state space `X` and time periods `t` as NumPy arrays.
- We initialize the value function matrix `V` with zeros. The first row of the matrix, `V[0, :]`, corresponds to the states where no inventory is left, and the first column of the matrix, `V[:, 0]`, corresponds to the states where no time is left. By initializing the entire matrix with zeros, the boundary conditions `V(0,t) = 0` for all `t` and `V(x,0) = 0` for all `x` are implicitly satisfied.
- In the iterative step, we calculate `V[x,t]` for each state `(x,t)` using equation (2). We discretize the price range and find the maximum value by comparing the immediate revenue plus the future value for each price.
- After calculating `V`, we extract the optimal bid price matrix `b_star` using equation (3) by taking the difference in value between adjacent inventory states.
- The `lambda_func` and `Pw_func` are example functions for the arrival rate and purchase probability, respectively. We can modify them based on your specific problem.
- Finally, we print the optimal bid price matrix `b_star`.

Note: This implementation assumes a discretized price range for simplicity. We can adjust the price range and granularity as needed.

In [ ]:
# Example usage
C_test = 3  # Capacity
T_test = 6  # Time horizon
delta_t_test = 1  # Time interval

V_test, b_star_test, p_star_test, p_star_alternative_test = optimal_bid_prices(C_test, T_test, 
                                                                               p0_func, pmax_func, lambda_func, Pw_func, delta_t_test)

# Print summary statistics
print("value function V: ", V_test)
print()
print("bid price: ", b_star_test)
# print("optimal price: ", p_star_test)
# print("optimal price using equation 6: ", p_star_alternative_test)
# print("optimal price difference", p_star_test - p_star_alternative_test)

---
---

## Proposed Method: Generate Bid Price using Data-Driven Approach - Neural Network Model

In the above implementation, we generate **synthetic historical data** of airline bookings. It includes information such as the time a booking was made, the price charged, total number of bookings for a single flight, total revenues, and load factor. The price and time data will be used to implement Observation Building process of the proposed methodology in the paper.

---

### Training Phase: Generate Historical Booking Data
In this subsection, we generate historical booking data using parallel processing. The goal is to speed up the computations.

In [ ]:
# from historical_data_utils import simulate_departure_training, optimal_bid_prices, p0_func, alpha_func, pmax_func, Pw_func, generate_historical_data
from historical_data_utils import simulate_departure_training, simulate_departure_testing_baseline, simulate_departure_testing_robustness 

**Historical Data**: Prices and time to departure for each departure/flight. $\lambda$ represents demand observed by the airline.

In [ ]:
def generate_historical_data(C, T, lambda_range, lambda_test_range, delta_t, num_simulations, num_departures, seed=None):
    # Set the seed for the random number generator
    if seed is not None:
        np.random.seed(seed)
    
    # Initialize historical data arrays
    prices = []
    times = []
    total_booked_seats = []
    load_factors = []
    total_revenues = []
    lambda_vals = []  # Store lambda training values for each baseline simulation
    lambda_test_vals = []  # Store lambda test values for each robustness simulation

    # Create a pool of worker processes
    pool = mp.Pool()
    # pool = mp.Pool(processes=num_cores)

    # Simulate historical data
    for i in range(num_simulations):
        # Randomly select the arrival rate (lambda) from the given range for each baseline simulation run
        lambda_val = np.random.uniform(lambda_range[0], lambda_range[1])
        lambda_vals.append(lambda_val)  # Store the lambda value for each simulation

        # Randomly select the arrival rate (lambda) from the given range for each robustness simulation run
        lambda_test_val = np.random.uniform(lambda_test_range[0], lambda_test_range[1])
        lambda_test_vals.append(lambda_test_val)  # Store the lambda test value for each simulation
        
        # Create a list of arguments for each departure simulation
        args_list = [(C, T, lambda_val, delta_t, seed+i*num_departures+j) for j in range(num_departures)]

        # Use the pool to simulate departures in parallel for training
        results = pool.map(simulate_departure_training, args_list)
        
        # Unpack the results
        simulation_prices, simulation_times, simulation_booked_seats = zip(*results)

        # Calculate load factors and revenues for each departure
        simulation_load_factors = [booked_seats / C for booked_seats in simulation_booked_seats]
        simulation_revenues = [sum(prices) for prices in simulation_prices]
        
        # Append data for each simulation
        prices.append(simulation_prices)
        times.append(simulation_times)
        total_booked_seats.append(simulation_booked_seats)
        load_factors.append(simulation_load_factors)
        total_revenues.append(simulation_revenues)

    # Close the pool
    pool.close()

    return prices, times, total_booked_seats, load_factors, total_revenues, lambda_vals, lambda_test_vals

In [ ]:
if __name__ == '__main__':
    # Define your variables here
    seed = 2024  # Set a seed value

    # Parameter values
    C = 50  # Capacity
    T = 150  # Time horizon
    delta_t = 1  # Time interval
    num_simulations = 10  # Number of simulations
    num_departures = 10 # Number of departure dates
    
    lambda_range = [2.4, 3.6]  # Range for arrival rate (lambda)
    lambda_test_range = [1.8, 3.6]  # Range for arrival rate (lambda)
    
    # Define data collection points (DCPs) with a one-day interval
    # dcps = list(range(T - 1, -1, -1))
    # days_prior = list(range(T))
    
    dcps = list(range(T, 0, -1))
    days_prior = list(range(1, T+1))
    
    # Measure execution time for parallel processing approach
    start_time = time.time()
    prices, times, total_booked_seats, load_factors, total_revenues, lambda_vals, lambda_test_vals = generate_historical_data(
        C, T, lambda_range, lambda_test_range, delta_t, num_simulations, num_departures, seed
    )
    end_time = time.time()
    execution_time = end_time - start_time
    
    # Print the execution times
    # print(f"Sequential approach execution time: {execution_time_seq:.2f} seconds")
    print(f"Parallel processing approach execution time: {execution_time:.2f} seconds")

In [ ]:
def observation_building(prices, times, capacity, flights, dcps):
    """
    Observation building for data-driven bid price generation.

    Args:
        prices (list): List of historical price vectors for each flight.
        times (list): List of time-to-departure vectors for each flight.
        capacity (int): Capacity bound.
        flights (int): Number of flights (i.e., departures)
        dcps (list): List of data collection points (DCPs).

    Returns:
        list: List of transformed price matrices for each flight.
    """
    transformed_prices = []

    for i in range(flights):
        price_matrix = []

        for dcp in dcps: # each row in the transformed_prices
            # Filter prices and times for the current DCP
            filtered_data = [(p, t) for p, t in zip(prices[i], times[i]) if t <= dcp]
            filtered_prices = [p for p, _ in filtered_data]

            # Sort filtered prices in descending order
            sorted_prices = sorted(filtered_prices, reverse=True)

            # Pad sorted prices with zeros up to the capacity bound
            padded_prices = sorted_prices + [0] * (capacity - len(sorted_prices))

            price_matrix.append(padded_prices)

        transformed_prices.append(price_matrix)

    return transformed_prices

Each row of `transformed_prices` represents a dcp in descending order and each column represents remaining capacity in ascending order.

### Prepare Data for Deep Neural Network Algorithm

In [ ]:
def prepare_data(transformed_prices, dcps):
    """
    Prepare input feature matrix X and target variable vector Y for the NN model.

    Args:
        transformed_prices (list): List of transformed price matrices for each flight.
        dcps (list): List of data collection points (DCPs).

    Returns:
        tuple: A tuple containing the input feature matrix X and target variable vector Y.
    """
    X = []
    Y = []

    for price_matrix in transformed_prices:
        num_dcps, num_prices = len(price_matrix), len(price_matrix[0])

        for i in range(num_dcps):
            for j in range(num_prices):
                X.append([dcps[i], j + 1])  # DCP and remaining capacity index (starting from 1)
                Y.append(price_matrix[i][j])  # Corresponding bid price proxy

    return X, Y

### Train NN Model

In [ ]:
def create_nn_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='softplus')
    ])
    return model

def train_nn_model(X, Y, epochs, batch_size, validation_split=0.2, verbose=0):
    input_shape = (X.shape[1],)
    model = create_nn_model(input_shape)

    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
                  loss='mse')

    X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=validation_split)

    model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,
              validation_data=(X_val, Y_val), verbose=verbose)

    return model

In [ ]:
# Initialize a list to store the transformed prices for each simulation
transformed_prices_all_simulations = []

# Iterate over each simulation
for sim_idx in range(num_simulations):
    # Get the historical prices and times for the current simulation
    sim_prices = prices[sim_idx]
    sim_times = times[sim_idx]

    # Apply observation building for the current simulation
    transformed_prices_sim = observation_building(sim_prices, sim_times, C, num_departures, dcps)
    
    # Append the transformed prices for the current simulation to the overall list
    transformed_prices_all_simulations.append(transformed_prices_sim)

In [ ]:
def train_bid_price_model(transformed_prices, dcps, epochs, batch_size):
    """
    Train a neural network model to estimate bid prices using DCPs and remaining capacity.

    Args:
        transformed_prices (list): List of transformed price matrices for each flight.
        dcps (list): List of data collection points (DCPs).
        epochs (int): Number of epochs for training.
        batch_size (int): Batch size for training.

    Returns:
        tuple: A tuple containing the trained model and the scaler used for normalization.
    """
    X, Y = prepare_data(transformed_prices, dcps)

    X = np.array(X)
    Y = np.array(Y)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    model = train_nn_model(X_scaled, Y, epochs=epochs, batch_size=batch_size)

    return model, scaler

def predict_bid_prices(model, scaler, dcps, capacity):
    """
    Predict bid prices using the trained model for all combinations of DCPs and remaining capacity.

    Args:
        model: Trained neural network model.
        scaler: Scaler used for normalization.
        dcps (list): List of data collection points (DCPs).
        capacity (int): Maximum remaining capacity.

    Returns:
        numpy.ndarray: Predicted bid prices for all combinations of DCPs and remaining capacity.
    """
    X_pred = []
    for dcp in dcps:
        for cap in range(1, capacity+1):
            X_pred.append([dcp, cap])
    X_pred = np.array(X_pred)

    X_pred_scaled = scaler.transform(X_pred)

    bid_prices = model.predict(X_pred_scaled)
    bid_prices = bid_prices.reshape(len(dcps), capacity)

    return bid_prices

In [ ]:
# Train the bid price NN model for each simulation
epochs = 100
batch_size = 128
bid_price_models = []
scalers = []
for sim_prices in transformed_prices_all_simulations:
    model, scaler = train_bid_price_model(sim_prices, dcps, epochs, batch_size)
    bid_price_models.append(model)
    scalers.append(scaler)

# Predict bid prices for each simulation
predicted_bid_prices_all_simulations = []
for model, scaler in zip(bid_price_models, scalers):
    predicted_bid_prices = predict_bid_prices(model, scaler, dcps, C)
    predicted_bid_prices_all_simulations.append(predicted_bid_prices)

In [ ]:
def interpolate_bid_prices(bid_prices, dcps, days_prior):
    """
    Interpolate bid prices between DCPs to get a separate bid price vector for each day prior to departure.

    Args:
        bid_prices (numpy.ndarray): Predicted bid prices for all combinations of DCPs and remaining capacity.
        dcps (list): List of data collection points (DCPs).
        days_prior (list): List of days prior to departure.

    Returns:
        list: Interpolated bid price vectors for each day prior to departure.
    """
    interpolated_bid_prices = []
    min_dcp = min(dcps)
    max_dcp = max(dcps)
    
    for day in days_prior:
        # If the day is present in dcps, which means there is direct correspondence between the day and a dcp
        if day in dcps:
            dcp_index = dcps.index(day) # get the index of dcp
            interpolated_bid_prices.append(bid_prices[dcp_index]) # Directly append the exact bid price
        else:
            if day < min_dcp:
                # If the day is smaller than the minimum DCP, use the bid price vector of the minimum DCP
                # For instance, day = 1, minimum dcp = 4
                interpolated_bid_prices.append(bid_prices[0])
            elif day > max_dcp:
                # If the day is greater than the maximum DCP, use the bid price vector of the maximum DCP
                # For instance, day = 300, maximum dcp = 299
                interpolated_bid_prices.append(bid_prices[-1])
            else:
                # Implement linear interpolation
                prev_dcp = max(dcp for dcp in dcps if dcp < day)
                next_dcp = min(dcp for dcp in dcps if dcp > day)
                prev_index = dcps.index(prev_dcp)
                next_index = dcps.index(next_dcp)
                # For instance, day = 45, prev_dcp = 44, next_dcp = 49
                # alpha = (45-44)/(49-44) = 1/5 = 0.2
                alpha = (day - prev_dcp) / (next_dcp - prev_dcp)
                # In the example above, current day is closer to prev_dcp, so bid price associated with prev_dcp gets higher weights.
                interpolated_prices = (1 - alpha) * bid_prices[prev_index] + alpha * bid_prices[next_index]
                interpolated_bid_prices.append(interpolated_prices)

    return interpolated_bid_prices

In [ ]:
predicted_bid_prices_all_simulations[0][0]

In [ ]:
# Interpolate bid prices for each simulation
interpolated_bid_prices_all_simulations = []
for predicted_bid_prices in predicted_bid_prices_all_simulations:
    interpolated_bid_prices = interpolate_bid_prices(predicted_bid_prices, dcps, days_prior)
    interpolated_bid_prices_all_simulations.append(interpolated_bid_prices)

In [ ]:
interpolated_bid_prices_all_simulations[0][0]

### Example: Get a bid price associated with a given time to departure and remaining capacity

In [ ]:
def get_bid_price(sim_index, days_prior_value, remaining_capacity, interpolated_bid_prices_all_simulations):
    """
    Get the bid price for a specific simulation scenario, days prior-to-departure, and remaining capacity.

    Args:
        sim_index (int): Index of the simulation scenario.
        days_prior_value (int): Days prior-to-departure value.
        remaining_capacity (int): Remaining capacity value.
        interpolated_bid_prices_all_simulations (list): List of interpolated bid price vectors for each simulation scenario.

    Returns:
        float: Bid price for the specified simulation scenario, days prior-to-departure, and remaining capacity.
    """
    # Check if the days_prior_value is valid
    if days_prior_value < 0 or days_prior_value >= len(interpolated_bid_prices_all_simulations[sim_index]):
        raise ValueError(f"Invalid days_prior_value: {days_prior_value}")

    # Check if the remaining_capacity is valid
    if remaining_capacity < 1 or remaining_capacity > len(interpolated_bid_prices_all_simulations[sim_index][days_prior_value]):
        raise ValueError(f"Invalid remaining_capacity: {remaining_capacity}")

    return interpolated_bid_prices_all_simulations[sim_index][days_prior_value][remaining_capacity - 1]

In [ ]:
# Get the bid price for simulation scenario 0, 10-15 days prior-to-departure, and remaining capacity of 5
# For any fixed remaining capacity, the estimated bid price increases as time-to-departure increases.
sim_index = 0
days_prior_value = [6, 7, 8, 9, 10, 11]
remaining_capacity = 3
for day in days_prior_value:
    bid_price = get_bid_price(sim_index, day, remaining_capacity, interpolated_bid_prices_all_simulations)
    print(f"Bid price for simulation {sim_index}, {day} days prior, and remaining capacity {remaining_capacity}: {bid_price}")

In [ ]:
# Get the bid price for simulation scenario 0, 10 days prior-to-departure, and remaining capacity of 3 to 9
# For any fixed time to departure, the estimated bid price increases as the remaining capacity decreases.
sim_index = 0
days_prior_value = 10
remaining_capacity = [1, 2, 3, 4]
for capacity in remaining_capacity:
    bid_price = get_bid_price(sim_index, days_prior_value, capacity, interpolated_bid_prices_all_simulations)
    print(f"Bid price for simulation {sim_index}, {days_prior_value} days prior, and remaining capacity {capacity}: {bid_price}")

### Visualize Estimated Bid Prices using NN Model

In [ ]:
def visualize_predicted_bid_prices(predicted_bid_prices, num_dcps, capacity):
    fig, ax = plt.subplots(figsize=(12, 6))
    fig.suptitle("Predicted Bid Prices", fontsize=16)

    im = ax.imshow(predicted_bid_prices.T, cmap='viridis', aspect='auto')
    ax.set_ylabel("Remaining Capacity")
    ax.set_xlabel("Days Before Departure")

    # Set y-axis ticks and labels
    y_ticks = np.arange(0, capacity, 1)  # Display ticks every 10 units
    ax.set_yticks(y_ticks)
    ax.set_yticklabels(y_ticks + 1)

    # Set x-axis ticks and labels
    x_ticks = np.arange(0, num_dcps, 3)  # Display ticks every 30 days
    ax.set_xticks(x_ticks)
    ax.set_xticklabels(num_dcps - x_ticks)

    cbar = ax.figure.colorbar(im, ax=ax)
    cbar.ax.set_ylabel("Bid Price", rotation=-90, va="bottom")

    plt.tight_layout()
    plt.show()

# Usage
num_dcps = len(dcps)
visualize_predicted_bid_prices(predicted_bid_prices_all_simulations[0], num_dcps, C)

### Visualize Bid Price Proxies for Each Flight

In [ ]:
def visualize_bid_price_proxies(bid_price_proxies, num_flights, num_dcps, capacity):
    num_rows = (num_flights - 1) // 4 + 1
    num_cols = min(num_flights, 4)

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(6 * num_cols, 6 * num_rows))
    # fig.suptitle("Bid Price Proxies", fontsize=16)

    # Find the minimum and maximum values across all bid price proxies
    vmin = np.min([np.min(proxies) for proxies in bid_price_proxies])
    vmax = np.max([np.max(proxies) for proxies in bid_price_proxies])

    for flight in range(num_flights):
        flight_bid_price_proxies = np.array(bid_price_proxies[flight])  # Convert to NumPy array
        
        row = flight // num_cols
        col = flight % num_cols
        ax = axes[row, col] if num_rows > 1 else axes[col]

        im = ax.imshow(flight_bid_price_proxies.T, cmap='viridis', aspect='auto', vmin=vmin, vmax=vmax)  # Transpose the matrix
        ax.set_title(f"Bid Price Proxies - Flight {flight + 1}")
        ax.set_xlabel("Days Before Departure")
        ax.set_ylabel("Remaining Capacity")

        # Set x-axis ticks and labels
        xticks = np.linspace(0, num_dcps - 1, 10, dtype=int)
        xticklabels = [num_dcps - x for x in xticks]
        ax.set_xticks(xticks)
        ax.set_xticklabels(xticklabels)

        # Set y-axis ticks and labels
        yticks = np.linspace(0, capacity, 10, dtype=int)
        yticklabels = [capacity - y for y in yticks]
        ax.set_yticks(yticks)
        ax.set_yticklabels(yticklabels)

    # Add a single colorbar for all subplots
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.ax.set_ylabel("Bid Price", rotation=-90, va="bottom")

    # Adjust the spacing between subplots and the colorbar
    fig.subplots_adjust(right=0.9, wspace=0.3, hspace=0.4)
    plt.show()

# Usage
num_flights = len(transformed_prices_all_simulations[0])
num_dcps = len(dcps)
visualize_bid_price_proxies(transformed_prices_all_simulations[0], num_flights, num_dcps, C)

### Figure 3: Plot lambda vs. average optimal load factor

In [ ]:
# Figure 3: Plot lambda vs. average optimal load factor
avg_load_factor = [statistics.mean(load_factors[i]) for i in range(len(load_factors))]

plt.figure(figsize=(8, 6))
plt.scatter(lambda_vals, avg_load_factor)
plt.xlabel('λ')
plt.ylabel('Average Optimal Load Factor')
plt.title('λ vs. Average Optimal Load Factor for 100 Simulation Scenarios')
plt.grid(True)
plt.show()

### Figure 4: Bid Price Proxy from Observations of 300-day Sample (lambda=3.4)

In [ ]:
def plot_bid_price_proxy(transformed_prices, predicted_bid_prices, optimal_dp_bid_prices, 
                         dbd_threshold, capacity, num_departures, total_time_horizon, colors=None):
    # Create a figure and axis for the plot
    fig, ax = plt.subplots(figsize=(8, 6))

    # Define default colors if not provided
    if colors is None:
        colors = plt.cm.viridis(np.linspace(0, 1, num_departures))

    # Iterate over each flight (departure)
    for flight_idx in range(num_departures):
        # Get the transformed prices (after applying Observation Building) for the current flight
        flight_prices = transformed_prices[flight_idx] # This is all the bid price proxies for the current flight

        # Filter the bid price proxies based on the dbd_threshold
        # If the total_time_horizon T=15, and dbd_threshold=10, then flight_prices[total_time_horizon-dbd_threshold:] = flight_prices[5:]
        # which is the last ten rows of the flight prices, or the flight prices for dbd <= 10.
        filtered_flight_prices = flight_prices[total_time_horizon-dbd_threshold:]
        # Flatten the 2D list flitered_flight_prices into 1D list filtered_bid_price_proxies
        filtered_bid_price_proxies = [price for prices in filtered_flight_prices for price in prices]
        
        # Create the remaining capacity array for the filtered bid price proxies
        remaining_capacity = [remaining_cap for _ in range(len(filtered_flight_prices)) for remaining_cap in range(1, capacity+1)]

        # Plot the filtered bid price proxies as scatter points
        ax.scatter(remaining_capacity, filtered_bid_price_proxies, color=colors[flight_idx], label=f'Flight {flight_idx+1}', alpha=0.7)

    # Filter the predicted bid prices based on the dbd_threshold
    filtered_predicted_bid_prices = predicted_bid_prices[total_time_horizon-dbd_threshold:, :]
    print(filtered_predicted_bid_prices.shape) # shape = (T, C)

    # Calculate the average predicted bid prices for each remaining capacity level
    avg_predicted_bid_prices = np.mean(filtered_predicted_bid_prices, axis=0)
    print(avg_predicted_bid_prices.shape) # Shape = (C, )

    # Plot the average predicted bid prices as a blue line
    remaining_capacity_levels = np.arange(1, capacity + 1)
    ax.plot(remaining_capacity_levels, avg_predicted_bid_prices, color='blue', label='Estimated Bid Prices')

    # Filter the optimal bid prices based on the dbd_threshold
    filtered_optimal_bid_prices = optimal_dp_bid_prices[:, :dbd_threshold]
    print(filtered_optimal_bid_prices.shape) # Shape = (C, T)

    # Calculate the average optimal bid prices for each remaining capacity level
    avg_optimal_bid_prices = np.mean(filtered_optimal_bid_prices, axis=1)
    print(avg_optimal_bid_prices.shape) # Shape = (C, )

    # Plot the optimal bid prices as a red line
    ax.plot(remaining_capacity_levels, avg_optimal_bid_prices, color='red', label='Optimal Bid Prices')

    # Set the x-axis label and limits
    ax.set_xlabel('Remaining Capacity')
    ax.set_xlim(0, capacity+1)

    # Set the y-axis label
    ax.set_ylabel('Bid Price Proxy')

    # Add a title indicating the dbd_threshold
    ax.set_title(f'Bid Price Proxy (Days Before Departure <= {dbd_threshold})')

    # Add a legend
    # ax.legend(title='Flight', loc='upper right', bbox_to_anchor=(1.2, 1), ncol=1, borderaxespad=0.)

    # Adjust the layout to make space for the legend
    plt.tight_layout()

    # Show the plot
    plt.show()

The list comprehension `[price for prices in filtered_flight_prices for price in prices]` involves three steps:

- Step 1: The outer loop `for prices in filtered_flight_prices` iterates through each sublist (row) in the list `filtered_flight_prices` and store each sublist in the variables called `prices`.
- Step 2: The inner loop `for price in prices` iterates through each element inside each sublist `prices` from step 1 and stores each element in the variable called `price`.
- Step 3: Apppend each element to the list. At the end, every element of the list `filtered_flight_prices` is flattened into 1D list.

In [ ]:
# Calculate the optimal bid prices
_, b_star, _, _ = optimal_bid_prices(C, T, p0_func, pmax_func, lambda_func, Pw_func, delta_t)
# Filter optimal bid prices for C>0 and T>0
optimal_dp_bid_prices = b_star[1:, 1:]

# Define the days before departure thresholds for each subplot
dbd_thresholds = [T, int(0.5*T), int(0.1*T)]

# Plot the bid price proxies and estimated bid prices for each dbd_threshold
for dbd_threshold in dbd_thresholds:
    plot_bid_price_proxy(transformed_prices_all_simulations[0], predicted_bid_prices_all_simulations[0], optimal_dp_bid_prices,
                         dbd_threshold, C, num_departures, T)

## Baseline Simulation and Results

In [ ]:
def generate_future_data_baseline(C, T, lambda_vals, delta_t, num_simulations, num_departures, optimal_bid_prices_all_simulations, interpolated_bid_prices_all_simulations, seed=None):
    # Set the seed for the random number generator
    if seed is not None:
        np.random.seed(seed)
    
    # Initialize future data arrays
    dp_load_factors_all = []
    dd_load_factors_all = []
    dp_revenues_all = []
    dd_revenues_all = []

    # Create a pool of worker processes
    pool = mp.Pool()

    # Simulate future data
    for i in range(num_simulations):
        # Get the lambda value for the current simulation
        lambda_val = lambda_vals[i]
        
        # Get the optimal and data-driven bid prices for the current simulation
        dp_bid_prices = optimal_bid_prices_all_simulations[i]
        dd_bid_prices = interpolated_bid_prices_all_simulations[i]
        
        # Create a list of arguments for each departure simulation
        args_list = [(C, T, lambda_val, delta_t, seed+i*num_departures+j) for j in range(num_departures)]

        # Use the pool to simulate departures in parallel
        results = pool.starmap(simulate_departure_testing_baseline, [(args, dp_bid_prices, dd_bid_prices) for args in args_list])
        
        # Unpack the results
        dp_prices, dp_departure_times, dd_prices, dd_departure_times, dp_load_factors, dd_load_factors, dp_revenues, dd_revenues = zip(*results)

        # Calculate average load factors and revenues for each simulation
        avg_dp_load_factor = np.mean(dp_load_factors)
        avg_dd_load_factor = np.mean(dd_load_factors)
        avg_dp_revenue = np.mean(dp_revenues)
        avg_dd_revenue = np.mean(dd_revenues)
        
        # Append data for each simulation
        dp_load_factors_all.append(avg_dp_load_factor)
        dd_load_factors_all.append(avg_dd_load_factor)
        dp_revenues_all.append(avg_dp_revenue)
        dd_revenues_all.append(avg_dd_revenue)

    # Close the pool
    pool.close()

    return dp_load_factors_all, dd_load_factors_all, dp_revenues_all, dd_revenues_all

In [ ]:
# Generate optimal bid prices and interpolated bid prices for each simulation
dp_bid_prices_all_simulations = []
dd_bid_prices_all_simulations = []
for i in range(num_simulations):
    # Create a new lambda_func that returns the constant value of lambda_vals[i]
    def lambda_func_constant(t):
        return lambda_vals[i]
        
    # Compute optimal bid prices using the optimal_bid_prices function
    _, dp_bid_prices, _, _ = optimal_bid_prices(C, T, p0_func, pmax_func, lambda_func_constant, Pw_func, delta_t)
    dp_bid_prices_all_simulations.append(dp_bid_prices)
    
    # Compute interpolated bid prices using the trained NN model
    dd_bid_prices = interpolate_bid_prices(predicted_bid_prices_all_simulations[i], dcps, days_prior)
    dd_bid_prices_all_simulations.append(dd_bid_prices)

# Generate future data
dp_load_factors_all, dd_load_factors_all, dp_revenues_all, dd_revenues_all = generate_future_data_baseline(C, T, lambda_vals, delta_t, num_simulations, num_departures, 
                                                                                                  dp_bid_prices_all_simulations, dd_bid_prices_all_simulations, seed)

In [ ]:
# Calculate revenue gap and load factor gap for each simulation
revenue_gaps = []
load_factor_gaps = []

for i in range(num_simulations):
    dp_revenue = dp_revenues_all[i]
    dd_revenue = dd_revenues_all[i]
    revenue_gap = (dd_revenue - dp_revenue) / dp_revenue
    revenue_gaps.append(revenue_gap)

    dp_load_factor = dp_load_factors_all[i]
    dd_load_factor = dd_load_factors_all[i]
    load_factor_gap = dd_load_factor - dp_load_factor
    load_factor_gaps.append(load_factor_gap)

In [ ]:
# Create Figure 5
plt.figure(figsize=(8, 6))
plt.scatter(dp_load_factors_all, revenue_gaps)
z = np.polyfit(dp_load_factors_all, revenue_gaps, 1)
p = np.poly1d(z)
plt.plot(dp_load_factors_all, p(dp_load_factors_all), "r--")
plt.xlabel('Average Optimal Load Factor')
plt.ylabel('Average Relative Revenue Gap from Optimal')
plt.title('Average Relative Revenue Gap vs. Average Optimal Load Factor')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=2))
plt.grid(True)
plt.show()

In [ ]:
# Create Figure 6
plt.figure(figsize=(8, 6))
plt.scatter(dp_load_factors_all, load_factor_gaps)
z = np.polyfit(dp_load_factors_all, load_factor_gaps, 2)
p = np.poly1d(z)
plt.plot(dp_load_factors_all, p(dp_load_factors_all), "r--")
plt.xlabel('Average Optimal Load Factor')
plt.ylabel('Average Load Factor Gap from Optimal')
plt.title('Average Load Factor Gap vs. Average Optimal Load Factor')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=2))
plt.grid(True)
plt.show()

---

## Robustness Simulation and Results
For all future arrival simulation for robustness check, the arrival rate is $\lambda_{test}$.

In [ ]:
def generate_future_data_robustness(C, T, lambda_train_vals, lambda_test_vals, 
                                    delta_t, num_simulations, num_departures, 
                                    optimal_bid_prices_all_simulations, benchmark_bid_prices_all_simulations, interpolated_bid_prices_all_simulations, 
                                    seed=None):
    # Set the seed for the random number generator
    if seed is not None:
        np.random.seed(seed)
    
    # Initialize future data arrays
    opt_load_factors_all = []
    bench_load_factors_all = []
    dd_load_factors_all = []
    opt_revenues_all = []
    bench_revenues_all = []
    dd_revenues_all = []

    # Create a pool of worker processes
    pool = mp.Pool()

    # Simulate future data
    for i in range(num_simulations):
        # Get the lambda values for the current simulation
        lambda_train_val = lambda_train_vals[i]
        lambda_test_val = lambda_test_vals[i]
        
        # Get the optimal, benchmark, and data-driven bid prices for the current simulation
        opt_bid_prices = optimal_bid_prices_all_simulations[i]
        bench_bid_prices = benchmark_bid_prices_all_simulations[i]
        dd_bid_prices = interpolated_bid_prices_all_simulations[i]
        
        # Create a list of arguments for each departure simulation
        args_list = [(C, T, lambda_test_val, delta_t, seed+i*num_departures+j) for j in range(num_departures)]

        # Use the pool to simulate departures in parallel
        results = pool.starmap(simulate_departure_testing_robustness, [(args, opt_bid_prices, bench_bid_prices, dd_bid_prices) for args in args_list])
        
        # Unpack the results
        opt_prices, opt_departure_times, bench_prices, bench_departure_times, dd_prices, dd_departure_times, opt_load_factors, bench_load_factors, dd_load_factors, opt_revenues, bench_revenues, dd_revenues = zip(*results)

        # Calculate average load factors and revenues for each simulation
        avg_opt_load_factor = np.mean(opt_load_factors)
        avg_bench_load_factor = np.mean(bench_load_factors)
        avg_dd_load_factor = np.mean(dd_load_factors)
        avg_opt_revenue = np.mean(opt_revenues)
        avg_bench_revenue = np.mean(bench_revenues)
        avg_dd_revenue = np.mean(dd_revenues)
        
        # Append data for each simulation
        opt_load_factors_all.append(avg_opt_load_factor)
        bench_load_factors_all.append(avg_bench_load_factor)
        dd_load_factors_all.append(avg_dd_load_factor)
        opt_revenues_all.append(avg_opt_revenue)
        bench_revenues_all.append(avg_bench_revenue)
        dd_revenues_all.append(avg_dd_revenue)

    # Close the pool
    pool.close()

    return opt_load_factors_all, bench_load_factors_all, dd_load_factors_all, opt_revenues_all, bench_revenues_all, dd_revenues_all

In [ ]:
# Generate optimal bid prices, benchmark bid prices and data-driven bid prices for each simulation
opt_bid_prices_all_simulations = []
bench_bid_prices_all_simulations = []
dd_bid_prices_all_simulations = []

for i in range(num_simulations):
    # Create lambda_func_train and lambda_func_test that return the constant values of lambda_train_vals[i] and lambda_test_vals[i]
    def lambda_func_train(t):
        return lambda_vals[i]
    
    def lambda_func_test(t):
        return lambda_test_vals[i]
        
    # Compute optimal bid prices using the optimal_bid_prices function with lambda_func_test
    _, opt_bid_prices, _, _ = optimal_bid_prices(C, T, p0_func, pmax_func, lambda_func_test, Pw_func, delta_t)
    opt_bid_prices_all_simulations.append(opt_bid_prices)
    
    # Compute benchmark bid prices (misspecified) using the optimal_bid_prices function with lambda_func_train
    _, bench_bid_prices, _, _ = optimal_bid_prices(C, T, p0_func, pmax_func, lambda_func_train, Pw_func, delta_t)
    bench_bid_prices_all_simulations.append(bench_bid_prices)
    
    # Compute interpolated bid prices using the trained NN model
    dd_bid_prices = interpolate_bid_prices(predicted_bid_prices_all_simulations[i], dcps, days_prior)
    dd_bid_prices_all_simulations.append(dd_bid_prices)

# Generate future data
opt_load_factors_all, bench_load_factors_all, dd_load_factors_all, opt_revenues_all, bench_revenues_all, dd_revenues_all = generate_future_data_robustness(
    C, T, lambda_vals, lambda_test_vals, delta_t, 
    num_simulations, num_departures, 
    opt_bid_prices_all_simulations, bench_bid_prices_all_simulations, dd_bid_prices_all_simulations, 
    seed
)

In [ ]:
# Calculate revenue gaps and load factor gaps for each simulation
revenue_gaps_bench = []
revenue_gaps_dd = []
load_factor_gaps_bench = []
load_factor_gaps_dd = []

for i in range(num_simulations):
    opt_revenue = opt_revenues_all[i]
    bench_revenue = bench_revenues_all[i]
    dd_revenue = dd_revenues_all[i]
    
    revenue_gap_bench = (bench_revenue - opt_revenue) / opt_revenue
    revenue_gap_dd = (dd_revenue - opt_revenue) / opt_revenue
    
    revenue_gaps_bench.append(revenue_gap_bench)
    revenue_gaps_dd.append(revenue_gap_dd)

    opt_load_factor = opt_load_factors_all[i]
    bench_load_factor = bench_load_factors_all[i]
    dd_load_factor = dd_load_factors_all[i]
    
    load_factor_gap_bench = bench_load_factor - opt_load_factor
    load_factor_gap_dd = dd_load_factor - opt_load_factor
    
    load_factor_gaps_bench.append(load_factor_gap_bench)
    load_factor_gaps_dd.append(load_factor_gap_dd)

### Figure 7 Number of Simulations per $\lambda_{test} / \lambda_{train}$ (Rounded to First Decimal)

In [ ]:
# Calculate the ratios of lambda_test_vals to lambda_vals
lambda_ratios = [lambda_test_vals[i] / lambda_vals[i] for i in range(num_simulations)]

# Round the ratios to one decimal place
lambda_ratios_rounded = [round(ratio, 1) for ratio in lambda_ratios]

# Create a dictionary to store the count of each rounded ratio
ratio_counts = {}
for ratio in lambda_ratios_rounded:
    if ratio not in ratio_counts:
        ratio_counts[ratio] = 0
    ratio_counts[ratio] += 1

# Create a list of unique rounded ratios and their corresponding counts
ratios = list(ratio_counts.keys())
counts = list(ratio_counts.values())

# Sort the ratios and counts based on the ratio values
ratios, counts = zip(*sorted(zip(ratios, counts)))

# Create a bar plot
fig, ax = plt.subplots(figsize=(8, 6))
ax.bar(ratios, counts, width=0.08, align='center', color='skyblue', edgecolor='black')

# Set the x-axis tick labels
ax.set_xticks(ratios)
ax.set_xticklabels(ratios)

# Set the labels and title
ax.set_xlabel(r'$\lambda_{test} / \lambda_{train}$', fontsize=12)
ax.set_ylabel('Number of Simulations', fontsize=12)
ax.set_title('Number of Simulations per $\lambda_{test} / \lambda_{train}$ (Rounded to First Decimal)', fontsize=14)

# Rotate the x-axis tick labels
plt.xticks(rotation=45)

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()

### Figure 8: Average Revenue Gap from Optimal w.r.t the Ratio of $\lambda_{test}$ to $\lambda_{train}$ for Benchmark and Data-Driven Simulations

In [ ]:
# Create a scatter plot for benchmark simulations
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(lambda_ratios, revenue_gaps_bench, color='red', label='Benchmark')

# Create a scatter plot for data-driven simulations
ax.scatter(lambda_ratios, revenue_gaps_dd, color='blue', label='Data-Driven')

# Calculate and plot trendlines
z_bench = np.polyfit(lambda_ratios, revenue_gaps_bench, 1)
p_bench = np.poly1d(z_bench)
ax.plot(lambda_ratios, p_bench(lambda_ratios), color='red', linestyle='--')

z_dd = np.polyfit(lambda_ratios, revenue_gaps_dd, 1)
p_dd = np.poly1d(z_dd)
ax.plot(lambda_ratios, p_dd(lambda_ratios), color='blue', linestyle='--')

# Set the labels and title
ax.set_xlabel(r'$\lambda_{test} / \lambda_{train}$', fontsize=12)
ax.set_ylabel('Average Revenue Gap from Optimal', fontsize=12)
ax.set_title('Average Revenue Gap from Optimal vs. $\lambda_{test} / \lambda_{train}$', fontsize=14)

# Format y-axis labels to be percentage
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=2))

# Add a legend
ax.legend()

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()

### Figure 9: Average Load Factor Gap from Optimal w.r.t the Ratio of $\lambda_{test}$ to $\lambda_{train}$ for Benchmark and Data-Driven Simulations

In [ ]:
# Create a scatter plot for benchmark simulations
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(lambda_ratios, load_factor_gaps_bench, color='red', label='Benchmark')

# Create a scatter plot for data-driven simulations
ax.scatter(lambda_ratios, load_factor_gaps_dd, color='blue', label='Data-Driven')

# Calculate and plot trendlines
z_bench = np.polyfit(lambda_ratios, load_factor_gaps_bench, 1)
p_bench = np.poly1d(z_bench)
ax.plot(lambda_ratios, p_bench(lambda_ratios), color='red', linestyle='--')

z_dd = np.polyfit(lambda_ratios, load_factor_gaps_dd, 1)
p_dd = np.poly1d(z_dd)
ax.plot(lambda_ratios, p_dd(lambda_ratios), color='blue', linestyle='--')

# Set the labels and title
ax.set_xlabel(r'$\lambda_{test} / \lambda_{train}$', fontsize=12)
ax.set_ylabel('Average Load Factor Gap from Optimal', fontsize=12)
ax.set_title('Average Load Factor Gap from Optimal vs. $\lambda_{test} / \lambda_{train}$', fontsize=14)

# Format y-axis labels to be percentage
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=2))

# Add a legend
ax.legend()

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()

### Figure 10: Ratio of Total Revenue and Average Load Factor of Data-Driven to Benchmark Simulation w.r.t the Ratio of $\lambda_{test}$ to $\lambda_{train}$

In [ ]:
# Calculate the revenue and load factor ratios
revenue_ratios = [dd_revenues_all[i] / bench_revenues_all[i] for i in range(num_simulations)]
load_factor_ratios = [dd_load_factors_all[i] / bench_load_factors_all[i] for i in range(num_simulations)]

# Create a scatter plot for revenue ratios
fig, ax1 = plt.subplots(figsize=(8, 6))
ax1.scatter(lambda_ratios, revenue_ratios, color='blue', label='Revenue Ratio')

# Calculate and plot trendline for revenue ratios
z_revenue = np.polyfit(lambda_ratios, revenue_ratios, 1)
p_revenue = np.poly1d(z_revenue)
ax1.plot(lambda_ratios, p_revenue(lambda_ratios), color='blue', linestyle='--')

# Set the labels and title for the first y-axis (revenue ratio)
ax1.set_xlabel(r'$\lambda_{test} / \lambda_{train}$', fontsize=12)
ax1.set_ylabel('Revenue Ratio (Data-Driven / Benchmark)', color='blue', fontsize=12)
ax1.tick_params('y', colors='blue')

# Create a second y-axis for load factor ratios
ax2 = ax1.twinx()
ax2.scatter(lambda_ratios, load_factor_ratios, color='red', label='Load Factor Ratio')

# Calculate and plot trendline for load factor ratios
z_load_factor = np.polyfit(lambda_ratios, load_factor_ratios, 1)
p_load_factor = np.poly1d(z_load_factor)
ax2.plot(lambda_ratios, p_load_factor(lambda_ratios), color='red', linestyle='--')

# Set the labels for the second y-axis (load factor ratio)
ax2.set_ylabel('Load Factor Ratio (Data-Driven / Benchmark)', color='red', fontsize=12)
ax2.tick_params('y', colors='red')

# Set the title
ax1.set_title('Revenue and Load Factor Ratios (Data-Driven / Benchmark) vs. $\lambda_{test} / \lambda_{train}$', fontsize=14)

# Add a legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()